### Importación de librerías

In [1]:
import cv2
import os
import mediapipe as mp

### Definición de variables

In [ ]:
DATASET_NAME = "../../dataset/gestos_cuerpo"
LABELS = {
    0: "0_Neutro",
    1: "1_Mano_Der_Arriba",      # Brazo levantado
    2: "2_Mano_Izq_Arriba",      # Brazo levantado
    3: "3_Punos_Cerrados",       # Ambas manos
    4: "4_Pulgar_Arriba",        # Gesto simple
    5: "5_Victoria",             # Gesto simple
    6: "6_Fuerza_Der",           # Bíceps derecho
    7: "7_Fuerza_Izq"            # Bíceps izquierdo
}

### Lógica de guardado de imágenes

In [ ]:
# Crear carpetas
if not os.path.exists(DATASET_NAME):
    os.makedirs(DATASET_NAME)

for key, nombre_carpeta in LABELS.items():
    path = os.path.join(DATASET_NAME, nombre_carpeta)
    if not os.path.exists(path):
        os.makedirs(path)

# === MEDIAPIPE HOLISTIC (MEJOR PARA BRAZOS + MANOS) ===
mp_holistic = mp.solutions.holistic
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles

# Usamos Holistic para ver hombros y codos también
holistic = mp_holistic.Holistic(
    static_image_mode=False,
    model_complexity=1,
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5
)

cap = cv2.VideoCapture(0)

# Contadores
contadores = {}
for key in LABELS:
    path = os.path.join(DATASET_NAME, LABELS[key])
    contadores[key] = len(os.listdir(path))

print(f"--- RECOLECTOR DE CUERPO Y MANOS ---")
print(f"Guardando en: {DATASET_NAME}")

while True:
    ret, frame = cap.read()
    if not ret: break
    
    # Espejo (importante para que 'Derecha' sea tu derecha en pantalla)
    frame = cv2.flip(frame, 1)  
    frame_display = frame.copy()
    
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = holistic.process(frame_rgb)
    
    cuerpo_detectado = False
    
    # Dibujar Pose (Brazos/Cuerpo)
    if results.pose_landmarks:
        cuerpo_detectado = True
        mp_drawing.draw_landmarks(
            frame_display,
            results.pose_landmarks,
            mp_holistic.POSE_CONNECTIONS,
            landmark_drawing_spec=mp_drawing_styles.get_default_pose_landmarks_style())

    # Dibujar Manos (Izquierda y Derecha)
    if results.left_hand_landmarks:
        mp_drawing.draw_landmarks(frame_display, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS)
    if results.right_hand_landmarks:
        mp_drawing.draw_landmarks(frame_display, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS)

    # === INTERFAZ ===
    fuente = cv2.FONT_HERSHEY_SIMPLEX
    
    # Panel superior (Resumen)
    info_counts = f"N:{contadores[0]} | MD_Up:{contadores[1]} | MI_Up:{contadores[2]} | Punos:{contadores[3]} | Vic:{contadores[5]}"
    cv2.putText(frame_display, info_counts, (10, 30), fuente, 0.5, (0, 255, 255), 1)

    # Panel inferior (Instrucciones)
    lineas = [
        "0:Neutro | 1:Mano Der Up | 2:Mano Izq Up",
        "3:Punos | 4:Pulgar | 5:Victoria",
        "6:Fuerza Der | 7:Fuerza Izq | q:Salir"
    ]
    
    y0 = 400
    for i, linea in enumerate(lineas):
        y = y0 + i * 25
        # Fondo negro para leer mejor
        (w, h), _ = cv2.getTextSize(linea, fuente, 0.6, 1)
        cv2.rectangle(frame_display, (10, y - h - 5), (10 + w + 10, y + 5), (0,0,0), -1)
        cv2.putText(frame_display, linea, (15, y), fuente, 0.6, (0, 255, 255), 1)

    if not cuerpo_detectado:
        cv2.putText(frame_display, "NO SE DETECTA CUERPO", (10, 200), fuente, 0.8, (0, 0, 255), 2)

    cv2.imshow('Recolector Cuerpo/Manos', frame_display)
    
    # === GUARDADO ===
    k = cv2.waitKey(1)
    if k == ord('q'): break
    
    clase = -1
    # Mapeo de teclas numéricas
    if 48 <= k <= 55: # ASCII del 0 al 7
        clase = k - 48
    
    if clase != -1:
        carpeta = LABELS[clase]
        idx = contadores[clase]
        path_img = os.path.join(DATASET_NAME, carpeta, f"{carpeta}_{idx}.jpg")
        cv2.imwrite(path_img, frame) # Guardamos frame limpio
        print(f"Guardado: {path_img}")
        contadores[clase] += 1

cap.release()
cv2.destroyAllWindows()

--- RECOLECTOR DE FOTOS ---
Las fotos se guardarán en la carpeta: ../dataset/gestos_mano
Foto guardada: ../dataset/gestos_mano\1_Puno\1_Puno_16.jpg
Foto guardada: ../dataset/gestos_mano\1_Puno\1_Puno_17.jpg
Foto guardada: ../dataset/gestos_mano\1_Puno\1_Puno_18.jpg
Foto guardada: ../dataset/gestos_mano\1_Puno\1_Puno_19.jpg
Foto guardada: ../dataset/gestos_mano\1_Puno\1_Puno_20.jpg
Foto guardada: ../dataset/gestos_mano\1_Puno\1_Puno_21.jpg
Foto guardada: ../dataset/gestos_mano\1_Puno\1_Puno_22.jpg
Foto guardada: ../dataset/gestos_mano\1_Puno\1_Puno_23.jpg
Foto guardada: ../dataset/gestos_mano\1_Puno\1_Puno_24.jpg
Foto guardada: ../dataset/gestos_mano\1_Puno\1_Puno_25.jpg
Foto guardada: ../dataset/gestos_mano\1_Puno\1_Puno_26.jpg
Foto guardada: ../dataset/gestos_mano\1_Puno\1_Puno_27.jpg
Foto guardada: ../dataset/gestos_mano\1_Puno\1_Puno_28.jpg
Foto guardada: ../dataset/gestos_mano\1_Puno\1_Puno_29.jpg
Foto guardada: ../dataset/gestos_mano\1_Puno\1_Puno_30.jpg
Foto guardada: ../dataset/